## Quick Abc to midi converter
- http://www.mandolintab.net/abcconverter.php


## Major and Minor
- https://www.youtube.com/watch?v=fKpUBsn_jmA
- help create mood and atmossphere of music

In [1]:
import os
import json
import numpy as np
import pandas as pd
import sys
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
BATCH_SIZE = 16
N_GRAM = 64
MODEL_DIR = './music_model/LSTMSkipConnection_many2many_128_256_128_layers_lstm/'
DATA_PATH = './data/jig_hornpipes_cleaned.txt'
MODEL_TYPE = 'Default'#unuse for now
BATCH_SIZE = 1
SEQ_LENGTH = 1

In [3]:
with open(os.path.join(MODEL_DIR, 'model_dictionary.json')) as f:
    vocab_map = json.load(f)

n_vocab = len(vocab_map['idx2char'])

print('n_vocab', n_vocab)

n_vocab 67


In [4]:
from models import MusicModel
# model = MusicModel(n_vocab).TwoLayerLSTM(batch_input_shape=(BATCH_SIZE, 1))
# model = MusicModel(n_vocab).TwoLayerLSTM(batch_input_shape=(BATCH_SIZE, SEQ_LENGTH))

# model = MusicModel(n_vocab).LayersRNNGeneric(batch_input_shape=(BATCH_SIZE, SEQ_LENGTH),
#                                             layers=['lstm', 'lstm', 'lstm'], 
#                                              layers_size=[128, 256, 128], drop_rate=0.4)

# model = MusicModel(n_vocab).LSTMSkipConnection(batch_input_shape=(BATCH_SIZE, SEQ_LENGTH),
#                                                                 layers=[128, 256, 128],
#                                                                 emb_dim=256,drop_rate=0.2)

model = MusicModel(n_vocab).LSTMSkipConnection(batch_input_shape=(BATCH_SIZE, SEQ_LENGTH),
                    layers=[128, 256, 128],
                    emb_dim=256,drop_rate=0.3)
print('\n',model.summary())
model.load_weights(MODEL_DIR+'/model_weight/'+"Weights_{}.h5".format(60))

Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (1, 1)               0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (1, 1, 256)          17152       input[0][0]                      
__________________________________________________________________________________________________
drop_layer1 (Dropout)           (1, 1, 256)          0           embedding[0][0]                  
__________________________________________________________________________________________________
lstm_layer0 (LSTM)              (1, 1, 128)          197120      drop_layer1[0][0]                
__________________________________________________________________________________________________
lstm_layer

In [54]:
def generate_sequence(model, vocab_map, initial_index=0, seq_length=300):
    sequence_index = [initial_index]
    n_vocab = len(vocab_map['idx2char'])
    prev_note = '\n'
    stop_note = ['\n', '%']
    for _ in range(seq_length):
        batch = np.zeros((1, 1))
        batch[0, 0] = sequence_index[-1]
#         print(batch)
        predicted_probs = model.predict_on_batch(batch).ravel()
#         print(predicted_probs.shape)
        
        sample = np.random.choice(range(n_vocab), size = 1, p = predicted_probs)
#         while vocab_map['idx2char'][str(sample[0])] in ['\n', '%']:
#         print(sample[0])
#         if  vocab_map['idx2char'][str(sample[0])] in ['\n','%', '\\']:
#             continue
        while vocab_map['idx2char'][str(sample[0])] in stop_note and prev_note in stop_note:
            sample = np.random.choice(range(n_vocab), size = 1, p = predicted_probs)
    
        prev_note = vocab_map['idx2char'][str(sample[0])]
        sequence_index.append(sample[0]) 
        
    str_seq = seq = ''.join(vocab_map['idx2char'][str(c)] for c in sequence_index)
    return str_seq
    

In [80]:
import random
initial_index = np.random.randint(0, n_vocab, size=(1,))[0]
music_abc = generate_sequence(model, vocab_map, initial_index=initial_index, seq_length=300)

print(music_abc)

4|"A"AAA eAA|"D"AdA fAa|"G"gab g3|"G"bge "D"agf|"E"gfe edB|
"A"A2A ABe|"A"ceA age|"A7"cea gfe|"D"d3 -d||
|
"E7"cde BdB|"Em"efe dcB|"A"ecA "A"Ace|"D"def "E7"e2=g|"A"aec a2:|
 [2"A"Ace ag=g||
"D"a2f a2f|"A"eac A2e|"Bm"gfe "B7"fed|"E7"e3 e2:|
P:B
f/2g/2|"A"aec aec|"D"d2b a2f|"A"e2c ABc|"D"d2e fga|
"A7"b


In [81]:
#header info: http://trillian.mit.edu/~jc/music/abc/doc/ABCtut_Headers.html#I_Key
header = '''
X: 1
T:AbcRnn
% abcRnn
M:4/4
K:K
'''

selected = '''
^D/2A/2B/2A/2A F2F/2G/2|
Aca aag|f/2a/2g/2f/2d/2 e3:|
\
A/2g/2f/2g/2|
f/2a3/2
e/2A/2 f/2g/2g/2f/2d/2^G/2|
"D"f2d/2d/2 "Bm"a3/2f/2a/2|"Em"g3 "E7/c+"e2d|"A7/c+"e2e "F/f+"d2c|"Bm"B2B A2^g|
"B7"a2f f2f|"E"g2e "B7"B2d|"B7"e2f g2g|"B7"a3 a2f|
"D7"d3 -c=cd|"G"B2G GBd|"C"e2e c2e|"Am"a3 "G7"g2d|"C"g3 z2:|
P:C


-----------------------------------------------------------------------------
_ef|"A"a^ag "D"fed|"F"ecA "G"Bcd|"Am"e3 -"D7"d2||
K:D
"D"AFA "G"Bcd|"D"Adf "G"gab|"D"agf def|"A7"edc A2a|"A7"afd e2c|
"D"d3 d2f|"D"A2f "A/e"c2e|"Bm"dcd "E7"e3|"A"cBc def|
"Bm"dfB BcB|"A"cde "E7"e2A|[1"A"A3 -A2||
 [2"A"A3 A2G/2F/2|E/2A/2e/2 "D"dcd|"A"e3 e2c|"A"A3 -A7||
K
K:D
"D"[2F2A "A"ABc|"Bm"d2d "D
'''

selected = '''
e|"G"d2e dBG|"Am"A2G "D"AFD|"G"G3 G2:|
P:B
d|"G"dBd def|"C"g2e "A7"e2e|"D7"f3 def|"G"g3 d2c|"Am"cdc "D7"A2c|"G"B2B B2e|"Am"d2d "D7"cBA|"G"G3 -G2||
A|"D"\
FDF F2A|"G"BAG "D"FAd|"A"c3 "F#m"ABc|"D"d3 -d2||
P:B
A|"D"FGA FGA|"D"FAA def|"G"g3 -g2f|"Em"edB "A7"ABc|
"D"d3 "A7"e3|"D"d3 d2c|"A"^cde A3|"A"^c2e 
'''



music_abc = header + music_abc

In [82]:
from music21 import *
abcScore = converter.parse(music_abc, format='abc')
mf = midi.translate.streamToMidiFile(abcScore)
mf.open('.//midi_epoch20-03.mid', 'wb')
mf.write()
mf.close()